In [ ]:
!pip install git+https://github.com/okankop/vidaug
!pip install scikit-image

In [ ]:
from vidaug import augmentors as va
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [ ]:
#initialize augmentator

sometimes05 = lambda aug: va.Sometimes(0.5, aug)

seq = va.Sequential([
    va.RandomRotate(degrees=35),  
    sometimes05(
        va.SomeOf([
            va.HorizontalFlip(),
            va.GaussianBlur(sigma=0.8),
            #va.ElasticTransformation(alpha=1, sigma=0),
        ], 2),
    ),
])

In [ ]:
# try on single example
v = np.load('output/highfive/0001.npy')
new_v = np.zeros(v.shape).astype('uint8')

#convert to RGB
for i in range(new_v.shape[0]):
    new_v[i] = cv2.cvtColor(v[i], cv2.COLOR_BGR2RGB)
    
video_aug = seq(new_v)
plt.imshow(video_aug[0])

In [ ]:
output_folder = 'output'

for class_n, c in enumerate(['kiss', 'handshake', 'hug', 'highfive']):
    class_folder = os.path.join(output_folder, c)
    f = os.listdir(class_folder)
    f = list(map(lambda x: os.path.join(class_folder, x), f))
    
    filtered = list(filter(lambda x: x.split('.')[-1] == 'npy', f))
    filtered.sort(key=lambda x: int(x.split('/')[-1].split('.')[0]))

    # next line commented so it doesn't use the augmented videos
    #last_number = int(filtered[-1].split('/')[-1].split('.')[-2].lstrip('0'))
    last_number = 50

    i = 1
    for f in filtered[:50]:
        v = np.load(f)
        for _ in range(3):
            video_aug = seq(v)
            filename = str(last_number+i).rjust(4, '0')
            np.save(os.path.join(class_folder, filename), video_aug)
            i += 1
    
    print(f'Class {c} (class_n) completed; additional {i} videos created')